# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse Shipments   Customer ShipmentStatus
0  Jan 16 2023  9:00AM  255058        12  HH-39380  Hush Hush       Released
1  Jan 16 2023  9:00AM  255057        12  HH-39381  Hush Hush       Released
2  Jan 16 2023  9:00AM  255057        12  HH-39382  Hush Hush       Released
3  Jan 16 2023  9:00AM  255057        12  HH-39383  Hush Hush       Released
4  Jan 16 2023  9:00AM  255057        12  HH-39384  Hush Hush       Released
5  Jan 16 2023  9:00AM  255057        12  HH-39385  Hush Hush       Released
6  Jan 16 2023  9:00AM  255057        12  HH-39386  Hush Hush       Released
7  Jan 16 2023  9:00AM  255057        12  HH-39387  Hush Hush       Released
8  Jan 16 2023  9:00AM  255057        12  HH-39388  Hush Hush       Released
9  Jan 16 2023  9:00AM  255057        12  HH-39389  Hush Hush       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
8   255055       Released          1
9   255056       Released          1
10  255057       Released         20
11  255058       Released          1
12  255059       Released          1

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
255055                NaN        NaN       1.0
255056                NaN        NaN       1.0
255057                NaN        NaN      20.0
255058                NaN        NaN       1.0
255059                NaN        NaN       1.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
255038                1.0        4.0       1.0
255044                0.0        1.0       1.0
255052                0.0        0.0      30.0
255053                0.0        0.0       1.0
255054                0.0        0.0       7.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
255038                  1          4         1
255044                  0          1         1
255052                  0          0        30
255053                  0          0         1
255054                  0          0         7

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               255038          1          4         1
1               255044          0          1         1
2               255052          0          0        30
3               255053          0          0         1
4               255054          0          0         7

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing  Released
0               255038         1         4         1
1               255044                   1         1
2               255052                            30
3               255053                             1
4               255054                             7

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                  Customer
0   Jan 16 2023  9:00AM  255058        12                 Hush Hush
1   Jan 16 2023  9:00AM  255057        12                 Hush Hush
21  Jan 16 2023  8:59AM  255059        10            CLINUVEL, Inc.
22  Jan 16 2023  8:58AM  255056        10            Eye Pharma Inc
23  Jan 16 2023  8:52AM  255055        19            Graystone, LLC
24  Jan 16 2023  8:46AM  255054        10         ISDIN Corporation
31  Jan 16 2023  8:44AM  255052        12                 Hush Hush
61  Jan 16 2023  8:42AM  255053        22         NBTY Global, Inc.
62  Jan 16 2023  8:30AM  255038        10         ISDIN Corporation
67  Jan 16 2023  8:30AM  255044        10  Senseonics, Incorporated

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                  Customer Completed  \
0  Jan 16 2023  9:00AM  255058        12                 Hush Hush             
1  Jan 16 2023  9:00AM  255057        12                 Hush Hush             
2  Jan 16 2023  8:59AM  255059        10            CLINUVEL, Inc.             
3  Jan 16 2023  8:58AM  255056        10            Eye Pharma Inc             
4  Jan 16 2023  8:52AM  255055        19            Graystone, LLC             
5  Jan 16 2023  8:46AM  255054        10         ISDIN Corporation             
6  Jan 16 2023  8:44AM  255052        12                 Hush Hush             
7  Jan 16 2023  8:42AM  255053        22         NBTY Global, Inc.             
8  Jan 16 2023  8:30AM  255038        10         ISDIN Corporation         1   
9  Jan 16 2023  8:30AM  255044        10  Senseonics, Incorporated             

  Executing  Released  
0                   1  
1                  20  
2                   1  
3                   1  
4                   1  
5                   7  
6                  30  
7                   1  
8         4         1  
9         1         1

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                  Customer  Released  \
0  Jan 16 2023  9:00AM  255058        12                 Hush Hush         1   
1  Jan 16 2023  9:00AM  255057        12                 Hush Hush        20   
2  Jan 16 2023  8:59AM  255059        10            CLINUVEL, Inc.         1   
3  Jan 16 2023  8:58AM  255056        10            Eye Pharma Inc         1   
4  Jan 16 2023  8:52AM  255055        19            Graystone, LLC         1   
5  Jan 16 2023  8:46AM  255054        10         ISDIN Corporation         7   
6  Jan 16 2023  8:44AM  255052        12                 Hush Hush        30   
7  Jan 16 2023  8:42AM  255053        22         NBTY Global, Inc.         1   
8  Jan 16 2023  8:30AM  255038        10         ISDIN Corporation         1   
9  Jan 16 2023  8:30AM  255044        10  Senseonics, Incorporated         1   

  Executing Completed  
0                      
1                      
2                      
3                      
4                      
5                      
6                      
7                      
8         4         1  
9         1

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse        Customer  Released Executing  \
0  Jan 16 2023  9:00AM  255058        12       Hush Hush         1             
1  Jan 16 2023  9:00AM  255057        12       Hush Hush        20             
2  Jan 16 2023  8:59AM  255059        10  CLINUVEL, Inc.         1             
3  Jan 16 2023  8:58AM  255056        10  Eye Pharma Inc         1             
4  Jan 16 2023  8:52AM  255055        19  Graystone, LLC         1             

  Completed  
0            
1            
2            
3            
4

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse        Customer  Released  Executing  \
0  Jan 16 2023  9:00AM  255058        12       Hush Hush         1        NaN   
1  Jan 16 2023  9:00AM  255057        12       Hush Hush        20        NaN   
2  Jan 16 2023  8:59AM  255059        10  CLINUVEL, Inc.         1        NaN   
3  Jan 16 2023  8:58AM  255056        10  Eye Pharma Inc         1        NaN   
4  Jan 16 2023  8:52AM  255055        19  Graystone, LLC         1        NaN   

   Completed  
0        NaN  
1        NaN  
2        NaN  
3        NaN  
4        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse        Customer  Released  Executing  \
0  Jan 16 2023  9:00AM  255058        12       Hush Hush         1        0.0   
1  Jan 16 2023  9:00AM  255057        12       Hush Hush        20        0.0   
2  Jan 16 2023  8:59AM  255059        10  CLINUVEL, Inc.         1        0.0   
3  Jan 16 2023  8:58AM  255056        10  Eye Pharma Inc         1        0.0   
4  Jan 16 2023  8:52AM  255055        19  Graystone, LLC         1        0.0   

   Completed  
0        0.0  
1        0.0  
2        0.0  
3        0.0  
4        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         1275251        11        5.0        1.0
12          765167        51        0.0        0.0
19          255055         1        0.0        0.0
22          255053         1        0.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  1275251        11        5.0        1.0
1        12   765167        51        0.0        0.0
2        19   255055         1        0.0        0.0
3        22   255053         1        0.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10        11        5.0        1.0
1        12        51        0.0        0.0
2        19         1        0.0        0.0
3        22         1        0.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse     Status  Value
0        10   Released   11.0
1        12   Released   51.0
2        19   Released    1.0
3        22   Released    1.0
4        10  Executing    5.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10    12   19   22
Status                         
Completed   1.0   0.0  0.0  0.0
Executing   5.0   0.0  0.0  0.0
Released   11.0  51.0  1.0  1.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10    12   19   22
0          Completed   1.0   0.0  0.0  0.0
1          Executing   5.0   0.0  0.0  0.0
2           Released  11.0  51.0  1.0  1.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10    12   19   22
0  Completed   1.0   0.0  0.0  0.0
1  Executing   5.0   0.0  0.0  0.0
2   Released  11.0  51.0  1.0  1.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()